<a href="https://colab.research.google.com/github/devroopsaha744/Bank-churn-prediction/blob/main/Kaggle_Playground_Season_04_EP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import binary_crossentropy
from sklearn.metrics import classification_report

In [32]:
train = pd.read_csv("/content/train.csv")

In [33]:
train

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3.0,0.00,2.0,1.0,0.0,181449.97,0.0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1.0,0.00,2.0,1.0,1.0,49503.50,0.0
2,2,15694510,Hsueh,678,France,Male,40.0,10.0,0.00,2.0,1.0,0.0,184866.69,0.0
3,3,15741417,Kao,581,France,Male,34.0,2.0,148882.54,1.0,1.0,1.0,84560.88,0.0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5.0,0.00,2.0,1.0,1.0,15068.83,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70503,70503,15702430,Ignatyeva,548,France,Female,44.0,10.0,0.00,1.0,1.0,1.0,31299.71,1.0
70504,70504,15756056,Pagnotto,541,France,Female,30.0,7.0,0.00,1.0,1.0,0.0,125877.22,0.0
70505,70505,15576928,Tu,651,France,Female,43.0,2.0,0.00,1.0,0.0,0.0,57762.00,1.0
70506,70506,15665110,Chidozie,509,France,Female,44.0,5.0,0.00,2.0,1.0,0.0,194310.10,0.0


In [34]:
train.isnull().sum()

id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                1
Tenure             1
Balance            1
NumOfProducts      1
HasCrCard          1
IsActiveMember     1
EstimatedSalary    1
Exited             1
dtype: int64

In [35]:
train = train.dropna()

In [36]:
train.shape

(70507, 14)

In [37]:
train.columns

Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [38]:
train.dtypes

id                   int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                float64
Tenure             float64
Balance            float64
NumOfProducts      float64
HasCrCard          float64
IsActiveMember     float64
EstimatedSalary    float64
Exited             float64
dtype: object

In [39]:
train = train.drop(['CustomerId'], axis = 1)

## **Exploratory Data Analysis**

In [40]:
train['Surname'].value_counts()

Hsia        1007
T'ien        956
Maclean      704
Hs?          701
Ts'ui        661
            ... 
Winn           1
Beck           1
Sokolova       1
Jacka          1
Lopez          1
Name: Surname, Length: 2528, dtype: int64

In [41]:
train = train.drop(['Surname'], axis = 1)

In [42]:
train['Geography'].value_counts()

France     40444
Spain      15299
Germany    14764
Name: Geography, dtype: int64

In [43]:
pd.crosstab(train['Geography'], train['Exited'])

Exited,0.0,1.0
Geography,,
France,33861,6583
Germany,9113,5651
Spain,12723,2576


In [44]:
train['Exited'].value_counts()

0.0    55697
1.0    14810
Name: Exited, dtype: int64

In [45]:
train = pd.get_dummies(train)

In [46]:
scl = MinMaxScaler()
scl_train = scl.fit_transform(train)
scl_train = pd.DataFrame(scl_train, columns = train.columns)

In [47]:
x = scl_train.drop(['Exited'], axis = 1)
y = scl_train['Exited']

In [48]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

## **Model Developement**

In [ ]:
clf1 = LogisticRegression()
clf2 = SVC()
clf3 = KNeighborsClassifier()
clf4 =  DecisionTreeClassifier()
clf5 = RandomForestClassifier()
clf6 = GradientBoostingClassifier()
clf7 = AdaBoostClassifier()

classifiers = [clf1, clf2, clf3, clf4, clf5, clf6, clf7]

for var in classifiers:
  var.fit(x_train, y_train)
  pred = var.predict(x_test)
  print(var.score(x_test, y_test))


0.7886025065027193
0.7908489004492788
0.7528375502482857
0.797469851028612
0.8596594939702057
0.864093166233152
0.8611965003546938


In [ ]:
#Using xgboost
xgb = XGBClassifier(n_estimators = 100)
xgb.fit(x_train, y_train)
xgb.score(x_test,y_test)

0.865513375950556

In [ ]:
y_pred = xgb.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92     26052
           1       0.73      0.57      0.64      6955

    accuracy                           0.87     33007
   macro avg       0.81      0.76      0.78     33007
weighted avg       0.86      0.87      0.86     33007



In [61]:
#Building a Neural Network using Keras API of tensorflow
ann = Sequential([
    Dense(units = 32, input_shape = (14,), activation = 'relu'),
    Dense(units = 32, activation = 'relu'),
    Dense(units = 1, activation = 'sigmoid')
])

In [62]:
ann.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 32)                480       
                                                                 
 dense_16 (Dense)            (None, 32)                1056      
                                                                 
 dense_17 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1569 (6.13 KB)
Trainable params: 1569 (6.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [63]:
ann.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [64]:
ann.fit(x_train, y_train, epochs = 30, batch_size =  64)

Epoch 1/30
882/882 [==============================] - 3s 2ms/step - loss: 0.4215 - accuracy: 0.8196
Epoch 2/30
882/882 [==============================] - 2s 3ms/step - loss: 0.3488 - accuracy: 0.8524
Epoch 3/30
882/882 [==============================] - 2s 2ms/step - loss: 0.3393 - accuracy: 0.8568
Epoch 4/30
882/882 [==============================] - 2s 2ms/step - loss: 0.3363 - accuracy: 0.8580
Epoch 5/30
882/882 [==============================] - 2s 2ms/step - loss: 0.3350 - accuracy: 0.8590
Epoch 6/30
882/882 [==============================] - 2s 2ms/step - loss: 0.3336 - accuracy: 0.8593
Epoch 7/30
882/882 [==============================] - 2s 2ms/step - loss: 0.3325 - accuracy: 0.8599
Epoch 8/30
882/882 [==============================] - 2s 2ms/step - loss: 0.3312 - accuracy: 0.8606
Epoch 9/30
882/882 [==============================] - 2s 2ms/step - loss: 0.3306 - accuracy: 0.8608
Epoch 10/30
882/882 [==============================] - 2s 3ms/step - loss: 0.3297 - accuracy: 0.8606

In [65]:
eval_res = ann.evaluate(x_test, y_test)
print(f"test loss : {eval_res[0]}")
print(f"test accuracy : {eval_res[1]}")

441/441 [==============================] - 1s 1ms/step - loss: 0.3204 - accuracy: 0.8655
test loss : 0.3204314708709717
test accuracy : 0.8654800653457642


In [66]:
y_pred1 = ann.predict(x_test)

441/441 [==============================] - 2s 4ms/step
